# Cash Flow Projection example

This notebook computes a simplified cash flow projection for a temporary retirement pension with a capital of €100.000,- that start in two years and stops in ten.

The simplified formula we'll use for computing the actuarial factor $F$ is:

$$
F_i = \sum_{t=0}^\infty \sum_{j=1}^n Q^{\textrm{cum}}_{ij}(t) P_j(t) D(t)
$$
where $Q^{\textrm{cum}}$ are the cumulative transition matrices, $P$ are the unit payments, and $D$ is the discounted value based on an interest curve. We'll assume a yearly interest of 5% and a constant mortality rate of 10%.

In [1]:
import numpy as np
import itertools as it

np.set_printoptions(precision=2)

## Unit payments

In [2]:
# Input unit payments
P = np.array([[0,0]] * 2 + [[1, 0]] * 8)
P

array([[0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

## Interest

In [3]:
# Input interest
i = np.array([1.05] * 10)
i

array([1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05, 1.05])

In [5]:
# Cumulative interest
# Naive approach
i_cum = [i[0]]
for t in range(len(i) - 1):
    i_cum.append(i[t + 1] * i_cum[t])
np.array(i_cum)

array([1.05, 1.1 , 1.16, 1.22, 1.28, 1.34, 1.41, 1.48, 1.55, 1.63])

In [6]:
# Cumulative interest
# Using a universal function!
i_cum = np.multiply.accumulate(i)
i_cum

array([1.05, 1.1 , 1.16, 1.22, 1.28, 1.34, 1.41, 1.48, 1.55, 1.63])

In [45]:
# Discounted value. Notice the implicit broadcasting and ufunc.
D = 1.0 / i_cum
D

array([0.95, 0.91, 0.86, 0.82, 0.78, 0.75, 0.71, 0.68, 0.64, 0.61])

## Transition probabilities

In [8]:
# Input mortality table (calendar years x ages)
m = np.array(100 * [100 * [0.1]])
m

array([[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]])

In [9]:
# Mortality rate for age 42
# Naive iterative approach.
q = []
for t in range(10):
    q.append(m[t, t + 42])
q

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [10]:
# Mortality rate for age 42
# Avoiding loops!
q = np.diagonal(m, 42)[:10]
q

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [11]:
# Surival rates
p = 1.0 - q

In [12]:
# Transition probablities matrices
# Don't construct arrays by hand!
q_11 = p
q_12 = 1.0 - p
q_21 = np.zeros_like(p)
q_22 = np.ones_like(p)
Q = np.column_stack([q_11, q_12, q_21, q_22]).reshape(-1, 2, 2)
Q

array([[[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]],

       [[0.9, 0.1],
        [0. , 1. ]]])

In [13]:
# Cumulative transition probabilities
np.matmul.accumulate(Q)

RuntimeError: Reduction not defined on ufunc with signature

In [44]:
# That didn't work. We can cheat by reducing the problem to scalar multiplication.
# Notice the fancy indexing to only select the upper-left element of all matrices.
np.multiply.accumulate(Q[::,0,0])

array([0.9 , 0.81, 0.73, 0.66, 0.59, 0.53, 0.48, 0.43, 0.39, 0.35])

In [43]:
# But we want full matrix multiplication. Let's do it by hand whilst avoiding a loop.
Q_cum = np.array(list(it.accumulate(Q, np.matmul)))
Q_cum

array([[[0.9 , 0.1 ],
        [0.  , 1.  ]],

       [[0.81, 0.19],
        [0.  , 1.  ]],

       [[0.73, 0.27],
        [0.  , 1.  ]],

       [[0.66, 0.34],
        [0.  , 1.  ]],

       [[0.59, 0.41],
        [0.  , 1.  ]],

       [[0.53, 0.47],
        [0.  , 1.  ]],

       [[0.48, 0.52],
        [0.  , 1.  ]],

       [[0.43, 0.57],
        [0.  , 1.  ]],

       [[0.39, 0.61],
        [0.  , 1.  ]],

       [[0.35, 0.65],
        [0.  , 1.  ]]])

## Actuarial factor

In [46]:
# Naively try to compute the cash flow
C = Q_cum @ P * D

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 2)

In [25]:
# That didn't work because NumPy doesn't know it should this element-wise for each time t. 
# So do tell it explicitly to do so:
C = np.array([Q_cum[t] @ P[t] * D[t] for t in range(10)])
C

array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.63, 0.  ],
       [0.54, 0.  ],
       [0.46, 0.  ],
       [0.4 , 0.  ],
       [0.34, 0.  ],
       [0.29, 0.  ],
       [0.25, 0.  ],
       [0.21, 0.  ]])

In [29]:
# We can loose the loop by using np.einsum()!
C = np.einsum("tij,tj,t->ti", Q_cum, P, D)
C

array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.63, 0.  ],
       [0.54, 0.  ],
       [0.46, 0.  ],
       [0.4 , 0.  ],
       [0.34, 0.  ],
       [0.29, 0.  ],
       [0.25, 0.  ],
       [0.21, 0.  ]])

In [30]:
# And the factor is:
F = C.sum(axis=0)
F

array([3.12, 0.  ])

## Expected payments

In [31]:
# Example capital of € 100.000,-.
# The yearly benefit is the capital divided by the actuarial factor.
B = 100_000 / F[0]
B

32012.155689580944

In [36]:
# Expected payments in the cash flow are undiscounted, so remove the discount.
forecast = B * np.einsum("ti,t->ti", C, 1 / D)
forecast

array([[    0.  ,     0.  ],
       [    0.  ,     0.  ],
       [23336.86,     0.  ],
       [21003.18,     0.  ],
       [18902.86,     0.  ],
       [17012.57,     0.  ],
       [15311.31,     0.  ],
       [13780.18,     0.  ],
       [12402.17,     0.  ],
       [11161.95,     0.  ]])

In [37]:
# Expected total payment:
forecast.sum(axis=0)

array([132911.08,      0.  ])